In [1]:
import pandas as pd, numpy as np
from datetime import datetime
# import yfinance as yf
import tushare as ts
import time, urllib
ts.set_token('8ef5ec61cdd848715c57c11d58dd71da1271f76b2420d2bac8aef123')
import glob
import os

In [10]:
def handle_data_files_return_tickers(file_path):
    ch_etfs_df = pd.read_csv(file_path)
#     ch_etfs_df.dropna(inplace = True)
    try:
        ch_etfs_df['基金规模\n[单位] 元'] = ch_etfs_df['基金规模\n[单位] 元'].apply(lambda x: float(x.replace(",","")))
        ch_etfs_df['机构投资者持有份额\n[报告期] 2019中报\n[单位] 份'] = ch_etfs_df['机构投资者持有份额\n[报告期] 2019中报\n[单位] 份'].apply(lambda x: float(x.replace(",","")))
        ch_etfs_df.sort_values("基金规模\n[单位] 元", ascending=False, inplace =True)
    except Exception as e:
        print(e)
    ch_etfs = ch_etfs_df.copy()
    ch_etfs['code'] =ch_etfs['证券代码'].apply(lambda x: str(x)[:6])
    ch_etfs_ticker = list(ch_etfs['code'].unique())
    return ch_etfs_ticker

def get_etf_data(Code, BeginTime, EndTime):
    df = ts.get_k_data(Code, index = False,  start = BeginTime, end = EndTime)
    return df


def download_tushare_etf_data(start, end, ch_db_path, ticker_list):
    count = 1
    for ticker in ticker_list:
        if count%200==0:
            print("=======================Sleeping======================")
            time.sleep(60)
        else:
            if not os.path.exists(ch_db_path+ticker+".csv"):
                print("{} is new, start downloading now...".format(ticker))
                try:
                    ############ replace get_data with other customized functions #############
                    data = get_etf_data(ticker, start, today)
                    ############ replace get_data with other customized functions #############
    #                 print(data)
                    data['date'] = data['date'].astype(str)
    #                 data['date'] = data['date'].apply(lambda x:x[:4]+"-"+x[4:6]+"-"+x[6:] if len(x)!=10 else x)
                    data.sort_values("date", inplace = True)
                    data.to_csv(ch_db_path+ticker+".csv", index = False)
                    print("{} data file created: {}".format(ticker, end))
                except Exception as e:
                    print(e)

            else:
                print("Already have data csv for {}".format(ticker))
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")   
                hist_data['date'] = hist_data['date'].astype(str)
                hist_data.to_csv(ch_db_path+ticker+".csv", index = False)
                hist_data = pd.read_csv(ch_db_path+ticker+".csv")
                try:
                    hist_data_last_date = hist_data['date'].values[-1]        
                    if today > hist_data_last_date:
                        print("Needs to update, start updating new data for {} now...".format(ticker))
                        update_start = hist_data_last_date
                        update_end = today
                        with eventlet.Timeout(60,False):
                            try:
                                ############ replace get_data with other customized functions #############
                                new_data = get_etf_data(ticker, update_start, update_end)
                                ############ replace get_data with other customized functions #############

                                new_data['date'] = new_data['date'].astype(str)
                                new_data.to_csv(ch_db_path+ticker+".csv", mode='a', header=False, index = False)
                                updated_duplicated_df = pd.read_csv(ch_db_path+ticker+".csv")
                                updated_df = updated_duplicated_df.drop_duplicates("date")
                                updated_df.sort_values("date", inplace = True)
                                updated_df.to_csv(ch_db_path+ticker+".csv", index = False)
                                print("New data updated till today for {}!".format(ticker))
                            except Exception as e:
                                print(e)
            #             print("Timed Out: Update Failed!")
                    else:
                        print("There's no new data to update for {}.".format(ticker))
                except Exception as e:
                    print(e)

        #     print("Data Download/Update for {} is Finished.".format(ticker))
            print("=======================Executed: {}=======================".format(count))
        count+=1
    print("【Updated Finished for today!】")

today = str(datetime.now().date())
start = '2010-01-01'
end = today
etf_ticker_path = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_ETFs.csv"
etf_path_to_csv = "/Users/miaoyuesun/Code_Workspace/brad_public_workspace_mac/data/CH_database/"
ticker_list = handle_data_files_return_tickers(etf_ticker_path)

'float' object has no attribute 'replace'


In [15]:
ticker_list

['518880',
 '518800',
 '515990',
 '515983',
 '515900',
 '515880',
 '515870',
 '515860',
 '515853',
 '515810',
 '515800',
 '515750',
 '515703',
 '515680',
 '515660',
 '515650',
 '515600',
 '515590',
 '515580',
 '515550',
 '515520',
 '515390',
 '515383',
 '515360',
 '515353',
 '515330',
 '515310',
 '515300',
 '515200',
 '515180',
 '515163',
 '515150',
 '515110',
 '515093',
 '515080',
 '515070',
 '515060',
 '515050',
 '515020',
 '515010',
 '515000',
 '513900',
 '513880',
 '513800',
 '513680',
 '513660',
 '513600',
 '513520',
 '513500',
 '513100',
 '513050',
 '513030',
 '513000',
 '512990',
 '512980',
 '512970',
 '512960',
 '512950',
 '512930',
 '512920',
 '512910',
 '512900',
 '512890',
 '512880',
 '512870',
 '512860',
 '512850',
 '512833',
 '512820',
 '512810',
 '512800',
 '512790',
 '512780',
 '512770',
 '512760',
 '512750',
 '512733',
 '512720',
 '512710',
 '512700',
 '512690',
 '512680',
 '512670',
 '512660',
 '512650',
 '512640',
 '512610',
 '512600',
 '512590',
 '512580',
 '512570',

In [16]:
get_etf_data("515983", start, end)

""


In [12]:
len(ticker_list)

261

In [4]:
import eventlet
eventlet.monkey_patch()

download_tushare_etf_data(start, end, etf_path_to_csv, ticker_list)

Already have data csv for 510500
Needs to update, start updating new data for 510500 now...
New data updated till today for 510500!
=======================Executed: 1=======================
Already have data csv for 510050
Needs to update, start updating new data for 510050 now...


/Users/miaoyuesun/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


New data updated till today for 510050!
=======================Executed: 2=======================
Already have data csv for 510300
Needs to update, start updating new data for 510300 now...
New data updated till today for 510300!
=======================Executed: 3=======================
Already have data csv for 510330
Needs to update, start updating new data for 510330 now...
New data updated till today for 510330!
=======================Executed: 4=======================
Already have data csv for 159919
Needs to update, start updating new data for 159919 now...
New data updated till today for 159919!
=======================Executed: 5=======================
Already have data csv for 510180
Needs to update, start updating new data for 510180 now...
New data updated till today for 510180!
=======================Executed: 6=======================
Already have data csv for 159915
Needs to update, start updating new data for 159915 now...
New data updated till today for 159915!
==========

New data updated till today for 159910!
=======================Executed: 45=======================
Already have data csv for 511270
Needs to update, start updating new data for 511270 now...
New data updated till today for 511270!
=======================Executed: 46=======================
Already have data csv for 513050
Needs to update, start updating new data for 513050 now...
New data updated till today for 513050!
=======================Executed: 47=======================
Already have data csv for 159948
Needs to update, start updating new data for 159948 now...
New data updated till today for 159948!
=======================Executed: 48=======================
Already have data csv for 512980
Needs to update, start updating new data for 512980 now...
New data updated till today for 512980!
=======================Executed: 49=======================
Already have data csv for 511020
Needs to update, start updating new data for 511020 now...
New data updated till today for 511020!
=====

New data updated till today for 512770!
=======================Executed: 88=======================
Already have data csv for 512700
Needs to update, start updating new data for 512700 now...
New data updated till today for 512700!
=======================Executed: 89=======================
Already have data csv for 159958
Needs to update, start updating new data for 159958 now...
New data updated till today for 159958!
=======================Executed: 90=======================
Already have data csv for 159965
Needs to update, start updating new data for 159965 now...
New data updated till today for 159965!
=======================Executed: 91=======================
Already have data csv for 510130
Needs to update, start updating new data for 510130 now...
New data updated till today for 510130!
=======================Executed: 92=======================
Already have data csv for 512920
Needs to update, start updating new data for 512920 now...
New data updated till today for 512920!
=====

New data updated till today for 510070!
=======================Executed: 131=======================
Already have data csv for 159913
Needs to update, start updating new data for 159913 now...
New data updated till today for 159913!
=======================Executed: 132=======================
Already have data csv for 512200
Needs to update, start updating new data for 512200 now...
New data updated till today for 512200!
=======================Executed: 133=======================
Already have data csv for 512780
Needs to update, start updating new data for 512780 now...
New data updated till today for 512780!
=======================Executed: 134=======================
Already have data csv for 159956
Needs to update, start updating new data for 159956 now...
New data updated till today for 159956!
=======================Executed: 135=======================
Already have data csv for 512810
Needs to update, start updating new data for 512810 now...
New data updated till today for 512810!
